In [1]:
from gee_fwi.FWI import FWICalculator
import ee
import eemont

from geemap import eefolium

In [2]:
ee.Initialize()

In [3]:
from datetime import date, datetime

# Forecast generated on 10 July 2021 06:00 UTC
forecast = datetime(2021, 7, 10, 6).strftime('%Y%m%d%H')
image_name = f'NOAA/GFS0P25/{forecast}F024'

# Fire Weather Index observation date
obs_date = date(2021, 7, 11)

In [4]:
# Get the 24 hours forecast from 10 July 2021 06 UTC
data = ee.Image(image_name)

# Image download boundaries
bounds = ee.Geometry.BBox(90, -15, 150, 15)

# Get temperature, relative humidity, and precipitation
rain = data.select('total_precipitation_surface').clip(bounds)
rhum = data.select('relative_humidity_2m_above_ground').clip(bounds)
temp = data.select('temperature_2m_above_ground').clip(bounds)

In [5]:
# Get wind speed
u_wind = data.select('u_component_of_wind_10m_above_ground').clip(bounds)
v_wind = data.select('v_component_of_wind_10m_above_ground').clip(bounds)
wind = (u_wind ** 2 + v_wind ** 2) ** 0.5

In [6]:
# Create a Google Earth Engine FWICalculator object
calculator = FWICalculator(obs_date, temp, rhum, wind, rain)

# Set initial values from a constant
calculator.set_initial_ffmc(85.0)
calculator.set_initial_dmc(6.0)
calculator.set_initial_dc(15.0)

In [7]:
# Calculate the Fire Weather Index
calculator.calculate_fire_weather_index()
fwi = calculator.fwi

In [8]:
# Visualize Indonesian Fire Weather Index
indonesia = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level0") \
    .filter(ee.Filter.eq('ADM0_NAME', 'Indonesia'))

vis_params = {
    'min' : 0,
    'max' : 13,
    'palette' : ['blue', 'green', 'yellow', 'red']    
}

Map = eefolium.Map(center = (0, 120), zoom = 5)
Map.addLayer(fwi.clip(indonesia), vis_params, 'Fire Weather Index')
Map